[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_2%5B4%5D.pdf)

In [1]:
import logging

In [2]:
logger = logging.getLogger("Algorithm info")
logger.setLevel(logging.INFO)
logger.disabled = True

In [3]:
import numpy as np
import pandas as pd
import math
import itertools as it
from common import *

norm = np.linalg.norm
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [4]:
class Point(np.ndarray):
    def __new__(cls, input_array):
        obj = np.array(input_array, dtype=float).view(cls)
        return obj
        
    def set_val(self, index, val):
        self[index] = val
        return self

pt = lambda *vals: Point(vals)

### [Wrapper counting fn. calls](https://stackoverflow.com/questions/1301735/counting-python-method-calls-within-another-method)

Zabava s Python anotacijama umjesto wrappanja u klasu.

In [5]:
def counted(fn):
    def wrapper(*args, **kwargs):
        wrapper.called += 1
        return fn(*args, **kwargs)
    wrapper.called = 0
    wrapper.__name__= fn.__name__
    return wrapper

def counts(fn):
    def wrapper(f, *args, **kwargs):
        if hasattr(f, 'called'):
            f.called = 0
            try:
                return fn(f, *args, **kwargs)
            finally:
                wrapper.calls_to_function = f.called
        else:
            return fn(f, *args, **kwargs)
    wrapper.__name__= fn.__name__
    return wrapper

## Funkcije
Funkcije i pripadne tocke

In [6]:
@counted
def f1(x):
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
f1_x0 = pt(-1.9, 2)
f1_xmin = pt(1, 1)
f1_min = 0

@counted
def f2(x):
    x1, x2 = x
    return (x1 - 4) ** 2 + 4 * (x2 - 2) ** 2
f2_x0 = pt(0.1, 0.3)
f2_xmin = pt(4, 2)
f2_min = 0

@counted
def f3(x):
    return np.sum(np.square(np.arange(1, x.shape[0] + 1) - x))
f3_x0 = pt(0, 0, 0, 0, 0)
f3_xmin = pt(1,2,3,4,5)
f3_min = 0

@counted
def f4(x):
    x1, x2 = x
    return abs((x1 - x2) * (x1 + x2)) + math.sqrt(x1 ** 2 + x2 ** 2)
f4_x0 = pt(5.1, 1.1)
f4_xmin = pt(0, 0)
f4_min = 0

@counted
def f6(x):
    x = np.sum(np.square(x))
    return 0.5 + (math.sin(math.sqrt(x)) ** 2 - 0.5) / (1 + 0.001 * x) ** 2
f6_xmin = 0
f6_min = 0

### [Golden section](http://www.fer.unizg.hr/_download/repository/zlatni_rez.txt)

In [7]:
@counts
def golden_section(f, a, b, e=1e-6, k=0.5*(math.sqrt(5) + 1)):
    hi = b - (b - a) / k
    lo = a + (b - a) / k
    
    f_hi = f(hi)
    f_lo = f(lo)
    
    while abs(b - a) > e:
        if f_hi < f_lo:
            b = lo
            lo = hi
            hi = b - (b - a) / k
            f_lo = f_hi
            f_hi = f(hi)
        else:
            a = hi
            hi = lo
            lo = a + (b - a) / k
            f_hi = f_lo
            f_lo = f(lo)
    
    return a, b

In [8]:
@counted 
def test_gs_fn(x): 
    return (x - 4.99)**2

print(golden_section(test_gs_fn, 0, 5))
print(golden_section.calls_to_function)

(4.989999525693381, 4.990000160114857)
35


### [Unimodal](http://www.fer.unizg.hr/_download/repository/unimodalni.txt)

In [9]:
@counts
def unimodal(f, x0, h=1):
    l, r = x0 - h, x0 + h
    m = x0
    step = 1
    
    fm = f(x0)
    fl = f(l)
    fr = f(r)
    
    if fm < fr and fm < fl:
        return l, r
    elif fm > fr:
        while fm > fr:
            logger.info("l = %s, r = %s, m = %s", l, r, m)
            logger.info("l = %f, r = %f, m = %f", fl, fr, fm)
            l, m, fm = m, r, fr
            r = x0 + h * step
            fr = f(r)
            step *= 2
    else:
        while fm > fl:
            logger.info("l = %s, r = %s, m = %s", l, r, m)
            logger.info("l = %f, r = %f, m = %f", fl, fr, fm)
            r, m, fm = m, l, fl
            l = x0 - h * step
            fl = f(l)
            step *= 2
    return l, r

In [10]:
unimodal(f1, pt(1, 1))

(Point([ 0.,  0.]), Point([ 2.,  2.]))

### Coordinate search

In [11]:
@counts
def coordinate_search(f, x0, eps=1e-6):
    x = Point(x0)
    x_prev = x + 2 * eps
    while norm(x_prev - x) > eps:
        x_prev = Point(x)
        for i in range(len(x)):
            f_in_single_dimension = lambda x_i_scalar_value: \
                f(x.set_val(i, x_i_scalar_value))
            l, r = unimodal(f_in_single_dimension, x[i])
            x[i] = np.mean(golden_section(
                f_in_single_dimension, l, r, eps))
    return x

### [Simplex](http://www.fer.unizg.hr/_download/repository/simplex.html)

In [12]:
class InvalidAlgorithmSetupException(Exception):
    def __init__(self, message):
        super(InvalidAlgorithmSetupException, self).__init__(message)
    
    def raiseIf(condition, message = "Invalid algorithm setup"):
        if condition:
            raise InvalidAlgorithmSetupException(message)

In [13]:
@counts
def nelder_mead(
    f, x0, step=1, alpha=1, beta=0.5, gamma=2, sigma=0.5, epsilon=1e-6, 
    max_iter=1000):
    InvalidAlgorithmSetupException.raiseIf(
        alpha <= 0 or gamma <= 1 or sigma <= 0 or sigma > 0.5)
    
    l, h = 0, -1
    x = sorted(simplex_pts(x0, step), key=f)
    xc = np.mean(x[:h], axis=0)
    
    reflect = lambda xc, xh: xc + alpha * (xc - xh)
    expand = lambda xc, xr: xc + gamma * (xr - xc)
    contract = lambda xc, xh: xc - beta * (xc - xh)
    shrink = lambda xl, xi: xl + sigma * (xi - xl)
    
    i = 0
    while norm(x[h] - xc) > epsilon and i < max_iter: 
        logger.info("xc = %s", xc)
        logger.info("f(xc) = %f", f(xc))
        
        xr = reflect(xc, x[h])
        
        fxr = f(xr)
        fxl = f(x[l])
        
        if fxr < fxl:
            xe = expand(xc, x[h])
            x[h] = xe if f(xe) < fxl else xr
        else:
            if all(fxr > f(xj) for xj in x[:h]):
                if fxr < f(x[h]):
                    x[h] = xr
                    
                xk = contract(xc, x[h])
                if f(xk) < f(x[h]):
                    x[h] = xk
                else:
                    shrink_xi = lambda xi: shrink(x[l], xi)
                    x = lmap(shrink_xi, x)
            else:
                x[h] = xr
        
        x = sorted(x, key=f)
        xc = np.mean(x[:h], axis=0)
        i += 1
            
    return Point(x[l])

def simplex_pts(x0, step):
    ret = [x0]
    for i in range(len(x0)):
        xp = Point(x0)
        xp[i] += step
        ret.append(xp)
    return np.array(ret)

In [14]:
nelder_mead(f1, f1_x0)

Point([ 0.99999976,  0.99999949])

### [Hooke-Jeeves](http://www.fer.unizg.hr/_download/repository/hj.html)

* x0 - pocetna tocka
* xB - bazna tocka 
* xP - pocetna tocka pretrazivanja
* xN - tocka dobivena pretrazivanjem

In [15]:
@counts
def hooke_jeeves(f, x0, epsilon=1e-6, dx=0.5):
    dx = dx * Point(np.ones_like(x0))
    epsilon = epsilon * Point(np.ones_like(x0))

    xp = Point(x0)
    xb = Point(x0)

    while all(dx > epsilon):
        xn = explore(f, xp, dx)
        fxn, fxb = f(xn), f(xb)
        
        logger.info("xb = %s", xb)
        logger.info("xp = %s", xp)
        logger.info("xn = %s", xn)
        logger.info("f(xn) = %f", fxn)
        logger.info("f(xb) = %f", fxb)
        
        if fxn < fxb:
            xp = 2 * xn - xb
            xb = Point(xn)
        else:
            dx /= 2.0
            xp = Point(xb)
        
    return xp

def explore(f, xp, dx):
    x = Point(xp)
    P = f(x)
    for i in range(len(xp)):
        x[i] = x[i] + dx[i]
        if f(x) > P:
            x[i] = x[i] - 2 * dx[i]
            if f(x) > P:
                x[i] = x[i] + dx[i]
    return x

In [16]:
hooke_jeeves(f1, f1_x0)

Point([ 1.00000153,  1.00000381])

# Zadaci
### 1. Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku pretraživanja postavite točku 10. Primijenite sva tri postupka na rješavanje ove funkcije te ispišite pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete zaključiti?

Neovisno o algoritmu i pocetnoj tocki, izgleda da svi u konacnici uspiju pronaci minimum. Zakljucak bi valjda onda bio da je funkcija konveksna i  stoga je jednostavno naci globalni minimum.

In [17]:
data = []
points = [ pt(10), pt(20), pt(30), pt(50), pt(100) ]
search_fs = [ hooke_jeeves, nelder_mead, coordinate_search ]

for f, x0 in it.product(search_fs, points):
    result = f(f3, x0)
    data.append([f.__name__, x0, f.calls_to_function, result])

pd.DataFrame(data, columns=['Algorithm', 'x0', 'Num calls', 'Result'])

,Algorithm,x0,Num calls,Result
0,hooke_jeeves,[10.0],132,[1.0]
1,hooke_jeeves,[20.0],147,[1.0]
2,hooke_jeeves,[30.0],158,[1.0]
3,hooke_jeeves,[50.0],179,[1.0]
4,hooke_jeeves,[100.0],222,[1.0]
5,nelder_mead,[10.0],236,[1.0]
6,nelder_mead,[20.0],296,[1.0]
7,nelder_mead,[30.0],356,[1.0]
8,nelder_mead,[50.0],476,[1.0]
9,nelder_mead,[100.0],776,[1.0]


## 2.
### Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim osima na funkcije 1-4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5). Za svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti iz rezultata?

In [18]:
fs = [f1, f2, f3, f4]
x0s = [f1_x0, f2_x0, f3_x0, f4_x0]
search_fs = [ nelder_mead, hooke_jeeves, coordinate_search ]

data = []
for search_f, (f, x0) in it.product(search_fs, zip(fs, x0s)):
    result = search_f(f, x0)
    data.append(
        [search_f.__name__, x0, search_f.calls_to_function, result])

pd.DataFrame(data, columns=['Algorithm', 'x0', 'Num calls', 'Result'])

,Algorithm,x0,Num calls,Result
0,nelder_mead,"[-1.9, 2.0]",4106,"[0.999999763998, 0.999999487937]"
1,nelder_mead,"[0.1, 0.3]",547,"[3.99999995768, 1.99999972735]"
2,nelder_mead,"[0.0, 0.0, 0.0, 0.0, 0.0]",2987,"[0.999999846924, 1.99999971796, 3.00000003086, 4.00000002405, 4.99999947917]"
3,nelder_mead,"[5.1, 1.1]",801,"[-8.3331812619e-07, 1.51895016504e-07]"
4,hooke_jeeves,"[-1.9, 2.0]",2952,"[1.00000152588, 1.0000038147]"
5,hooke_jeeves,"[0.1, 0.3]",260,"[3.99999961853, 2.00000076294]"
6,hooke_jeeves,"[0.0, 0.0, 0.0, 0.0, 0.0]",342,"[1.0, 2.0, 3.0, 4.0, 5.0]"
7,hooke_jeeves,"[5.1, 1.1]",150,"[3.1, 3.1]"
8,coordinate_search,"[-1.9, 2.0]",203969,"[0.999731862994, 0.999463734885]"
9,coordinate_search,"[0.1, 0.3]",354,"[4.00000015155, 2.00000004406]"


### 3. Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku (5, 5). Objasnite rezultate!

Hooke Jeeves ostane u lokalnom minimumu, dok Nelder Mead uspije pronaci globalni.

In [19]:
hooke_jeeves(f4, pt(5, 5))

Point([ 5.,  5.])

In [20]:
nelder_mead(f4, pt(5, 5))

Point([ -8.84859430e-08,  -3.86806220e-07])

In [21]:
f4_xmin

Point([ 0.,  0.])

### 4. Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5). Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment. Što možete zaključiti?

Algoritam se puno bolje ponasa kada za pocetnu tocku uzmemo neku koja je blizu stvarnog minimuma, i to neovisno o koraku koji odaberemo.

In [22]:
data = []
x0s = [ pt(0.5, 0.5), pt(20, 20) ]

for step, x0 in it.product(range(1, 21), x0s):
    result = nelder_mead(f1, x0, step=step, max_iter=5000)
    data.append([x0, 
                 step, 
                 nelder_mead.calls_to_function, 
                 result, 
                 f1(result)])
        
pd.DataFrame(data, columns=['x0', 'Step', 'Num calls', 'Result', 'Cost'])

,x0,Step,Num calls,Result,Cost
0,"[0.5, 0.5]",1,463,"[1.00000663406, 1.00001278068]",6.777450e-11
1,"[20.0, 20.0]",1,37015,"[1.39821731206, 1.95548991347]",1.585999e-01
2,"[0.5, 0.5]",2,5386,"[0.99999989005, 0.999999747216]",1.202295e-13
3,"[20.0, 20.0]",2,36467,"[1.41380822472, 1.99741612468]",1.714439e-01
4,"[0.5, 0.5]",3,3310,"[1.00000161348, 1.00000318316]",2.795237e-12
5,"[20.0, 20.0]",3,36549,"[1.03741465643, 1.07615287791]",1.400439e-03
6,"[0.5, 0.5]",4,1907,"[0.999999272771, 0.999998478111]",9.835678e-13
7,"[20.0, 20.0]",4,36715,"[1.23143622404, 1.51688359406]",5.358283e-02
8,"[0.5, 0.5]",5,1405,"[1.00000047531, 1.00000095745]",2.305811e-13
9,"[20.0, 20.0]",5,36602,"[1.37599661186, 1.89285910181]",1.413992e-01


### 5. Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni minimum ako je nađena vrijednost funkcije cilja manja od 10^-4)

Iz ovog uzorka, izgleda da je vjerojatnost da moja implementacija nadje minimum jednaka nuli. :)

In [23]:
ncalls = 200
threshold = 1e-4

data = []
for i in range(ncalls):
    sign = np.sign(np.random.rand(2) - 0.5)
    random = sign * np.random.rand(2) * 50
    result = nelder_mead(f6, pt(*random))
    cost = f6(result)
    
    data.append([random,
                 hooke_jeeves.calls_to_function, 
                 result,
                 cost,
                 cost < threshold ])

df_columns = ['x0', 'Num calls', 'Result', 'Cost', 'Min']
df = pd.DataFrame(data, columns=df_columns)
print('Nasao globalni optimum:', np.any(df['Min']))
df

Nasao globalni optimum: False


,x0,Num calls,Result,Cost,Min
0,"[35.3227211178, 28.7024754516]",133,"[37.4065503734, 28.6362081242]",0.4517757004684931,False
1,"[-36.2686437482, 46.9865464266]",133,"[-36.1984980354, 47.4450690652]",0.4759767606843276,False
2,"[-21.4181350542, 38.1690224492]",133,"[-21.241805611, 38.495586694]",0.4419083212442703,False
3,"[47.3375954617, -5.54588570793]",133,"[46.8740652971, -4.70153463573]",0.4517757004684931,False
4,"[37.8972752255, 18.3942849742]",133,"[40.0341627317, 18.1766419921]",0.4419083212442703,False
5,"[-27.8110405154, 21.749824794]",133,"[-26.8502082335, 21.730174859]",0.3960979202709431,False
6,"[30.1764336201, -46.9335826639]",133,"[30.9018705736, -47.3424019757]",0.47161465927523405,False
7,"[46.7285249803, -34.9909257698]",133,"[45.521111684, -33.5269596159]",0.47161465927523405,False
8,"[44.5633143102, -41.6040641863]",133,"[44.0607428196, -40.2494321038]",0.4759767606843276,False
9,"[-17.8660942056, 20.1385958853]",133,"[-18.1509936384, 21.6585188622]",0.3455063437531497,False


# Matlab
* `fminbnd` (1 var)
* `fminsearch` (vise var; simplex po N&M)
* `fminunc` (vise var; razliciti alg.)
* `fmincon` (vise var, uz ogranicenja)